In [1]:
import json
import random
import time

from faker import Faker
from confluent_kafka import SerializingProducer
from datetime import datetime

In [2]:
fake = Faker()

def generate_sales_transactions():
    user = fake.simple_profile()

    return {
        "transactionId": fake.uuid4(),
        "productId": random.choice(['1', '2', '3', '4', '5', '6', '7', '8', '9']),
        "productName": random.choice(['laptop', 'mobile', 'tablet', 'watch', 'headphone', 'speaker', 'PC', 'tablet', 'keyboard']),
        'productCategory': random.choice(['electronic', 'fashion', 'grocery', 'home', 'beauty', 'sports', 'gaming', 'casual', 'competitive']),
        'productPrice': round(random.uniform(10, 1000), 2),
        'productQuantity': random.randint(1, 10),
        'productBrand': random.choice(['apple', 'samsung', 'oneplus', 'mi', 'boat', 'sony', 'microsoft', 'nintendo', 'huawei']),
        'currency': random.choice(['USD', 'GBP']),
        'customerId': user['username'],
        'transactionDate': datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.%f%z'),
        "paymentMethod": random.choice(['credit_card', 'debit_card', 'online_transfer'])
    }

In [3]:
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f"Message delivered to {msg.topic} [{msg.partition()}]")

In [4]:
def main():
    topic = 'financial_transactions'
    producer= SerializingProducer({
        'bootstrap.servers': 'localhost:9092'
    })

    curr_time = datetime.now()

    while (datetime.now() - curr_time).seconds < 120:
        try:
            transaction = generate_sales_transactions()
            transaction['totalAmount'] = transaction['productPrice'] * transaction['productQuantity']

            print(transaction)

            producer.produce(topic,
                             key=transaction['transactionId'],
                             value=json.dumps(transaction),
                             on_delivery=delivery_report
                             )
            producer.poll(0)
            
            # Wait for 5 seconds before sending the next transaction
            time.sleep(5)
        except BufferError:
            print("Buffer full! Waiting...")
            time.sleep(1)
        except Exception as e:
            print(e)

In [5]:
if __name__ == "__main__":
    main()

{'transactionId': '11119980-bf95-434b-a17f-cc67303c48cd', 'productId': '8', 'productName': 'speaker', 'productCategory': 'home', 'productPrice': 762.77, 'productQuantity': 3, 'productBrand': 'mi', 'currency': 'USD', 'customerId': 'michael54', 'transactionDate': '2024-10-26T12:25:14.371948', 'paymentMethod': 'credit_card', 'totalAmount': 2288.31}
{'transactionId': '79496512-d5e0-4dcd-9cfc-80128ac9b553', 'productId': '6', 'productName': 'watch', 'productCategory': 'grocery', 'productPrice': 64.13, 'productQuantity': 8, 'productBrand': 'boat', 'currency': 'USD', 'customerId': 'lisabaker', 'transactionDate': '2024-10-26T12:25:19.373155', 'paymentMethod': 'credit_card', 'totalAmount': 513.04}
Message delivered to <built-in method topic of cimpl.Message object at 0x0000024E21930940> [0]
{'transactionId': 'd4d7c431-5b2d-4ee4-afc5-02f4f38264cf', 'productId': '8', 'productName': 'tablet', 'productCategory': 'gaming', 'productPrice': 133.47, 'productQuantity': 5, 'productBrand': 'samsung', 'curr

KeyboardInterrupt: 